<a href="https://colab.research.google.com/github/VincentZuo/Code/blob/main/20230819_web_app_using_fastapi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U click

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.1.6
    Uninstalling click-8.1.6:
      Successfully uninstalled click-8.1.6


In [2]:
!pip install fastapi nest-asyncio pyngrok uvicorn colabcode
!pip install git+https://github.com/ml-research/safe-latent-diffusion &> /dev/null
!pip install ftfy &> /dev/null
!curl -LJO https://github.com/crowsonkb/simulacra-aesthetic-models/blob/master/models/sac_public_2022_06_29_vit_b_16_linear.pth?raw=true
!pip install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio==0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!git clone https://github.com/openai/CLIP.git
!cd CLIP
!pip install .
!cd ..

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 85.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of nbclassic to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━

In [14]:
from pyngrok import ngrok

!ngrok authtoken 1XtjSbpm2hB1w98nOBegDtEP1sc_354W2yCfkWRnCPozMqBBp

ngrok.kill()

# !huggingface-cli login

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [15]:
!killall ngrok

ngrok: no process found


In [23]:
%%writefile index.html
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Stable Diffusion Model</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            padding: 20px;
            background-color: #f4f4f4;
        }

        h1 {
            text-align: center;
            color: #333;
        }

        form {
            background-color: #fff;
            padding: 20px;
            border-radius: 5px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        }

        label {
            font-weight: bold;
        }

        textarea, input[type="number"], input[type="button"] {
            width: 100%;
            padding: 10px;
            margin: 10px 0;
            border-radius: 5px;
            border: 1px solid #ddd;
        }

        input[type="button"] {
            background-color: #333;
            color: #fff;
            cursor: pointer;
        }

        input[type="button"]:hover {
            background-color: #555;
        }

        #big {
            margin-top: 20px;
        }

        img {
            max-width: 100%;
            border: 1px solid #ddd;
            border-radius: 5px;
            margin-top: 10px;
        }

        p {
            margin-top: 10px;
        }

    </style>
</head>

<body>
    <h1>Stable Diffusion Model</h1>
    <div>
        <form method="post" id="myForm">
            <label for="prompt">Prompt</label><br>
            <textarea id="prompt" name="prompt" rows="4" cols="50" placeholder="Enter String"></textarea><br>
            <label for="numbers">Enter count of images</label><br>
            <input type="number" id="numbers" name="N" min="1"><br>
            <input type="button" value="Start" onclick="submitForm()">
        </form>
        <div id="big"></div>
    </div>
    <script>
         function submitForm() {
             var formElement = document.getElementById('myForm');
             var data = new FormData(formElement);
             var list = document.getElementById("big")
             while (list.hasChildNodes()) {
                list.removeChild(list.firstChild);
              }
             var loading = document.createElement("h4")
             loading.id = "loading"
             loading.innerHTML = "Generating images..... Please wait "
             document.getElementById("big").appendChild(loading)
             fetch('calculate', {
                   method: 'POST',
                   headers: {
                     'Content-Type': 'application/json'
                    },
                   body: JSON.stringify(Object.fromEntries(data))
                 })
                 .then(resp => resp.json())  // or, resp.json(), etc.
                 .then(data => {
                   data = JSON.parse(data);

                   document.getElementById("loading").remove()

                   var responses = document.createElement("div") ;
                   responses.id = "responses"
                   document.getElementById("big").appendChild(responses)

                   var i =0


                   for(; i < data.length;i++)
                   {
                     var div = document.createElement("div");
                     div.id = "div" + i.toString()
                     document.getElementById("responses").appendChild(div)

                     var image = document.createElement("img")
                     image.src = data[i]['image']
                     document.getElementById(div.id).appendChild(image)

                     var score = document.createElement("p") ;
                     score.innerHTML = data[i]['score']
                     document.getElementById(div.id).appendChild(score)
                   }
                   var para = document.createElement("h3")
                   para.innerHTML = "Best image\n"
                   document.getElementById("big").appendChild(para)

                   var div = document.createElement("div");
                   div.id = "div" +  i.toString() ;
                   document.getElementById("big").appendChild(div)

                   var image = document.createElement("img")
                   image.src = data[0]['image']
                   document.getElementById(div.id).appendChild(image)

                   var score = document.createElement("p") ;
                   score.innerHTML = data[0]['score']
                   document.getElementById(div.id).appendChild(score)


                 })
                 .catch(error => {
                   console.error(error);
                 });
         }
    </script>
</body>

</html>


Overwriting index.html


In [17]:
from sld import SLDPipeline
import torch
stable_diffusion_src_path = "CompVis/stable-diffusion-v1-4"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

with torch.no_grad() :
  pipe = SLDPipeline.from_pretrained(
      stable_diffusion_src_path,
      #safety_checker = None,
  )
path= 'model.pth'
torch.save(pipe  , path )

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [19]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchvision import transforms
from torchvision.transforms import functional as TF
from CLIP import clip
from tqdm import tqdm
from PIL import Image

path = 'model.pth'

class AestheticMeanPredictionLinearModel(nn.Module):
    def __init__(self, feats_in):
        super().__init__()
        self.linear = nn.Linear(feats_in, 1)

    def forward(self, input):
        x = F.normalize(input, dim=-1) * input.shape[-1] ** 0.5
        return self.linear(x)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

clip_model_name = 'ViT-B/16'
clip_model = clip.load(clip_model_name, jit=False, device=device)[0]
clip_model.eval().requires_grad_(False)

asthetic_model = AestheticMeanPredictionLinearModel(512)
asthetic_model.load_state_dict(
    torch.load("/content/sac_public_2022_06_29_vit_b_16_linear.pth?raw=true")
)
asthetic_model = asthetic_model.to(device)
gen_model = torch.load(path , map_location = device)

In [20]:
async def getImages(model , text , count) :

  if model :
    gen = torch.Generator(device=device)
    gen.manual_seed(3503638049)
    print(".................Generating images ...............")
    with torch.no_grad() :
      out = model(prompt= text, generator=gen, guidance_scale=1 , num_images_per_prompt =  count )

    return out[0]

async def scoreImages(asthetic_model , clip_model   , images )  :
    print("............Scoring images.................")
    normalize = transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                                  std=[0.26862954, 0.26130258, 0.27577711])

    scores = []

    for image in tqdm(images):
        img = image.convert('RGB')
        img = TF.resize(img, 224, transforms.InterpolationMode.LANCZOS)
        img = TF.center_crop(img, (224,224))
        img = TF.to_tensor(img).to(device)
        img = normalize(img)
        clip_image_embed = F.normalize(
            clip_model.encode_image(img[None, ...]).float(),
            dim=-1)
        score = asthetic_model(clip_image_embed)
        scores.append((score.item(),image))


    scores.sort(key=lambda x: x[0])

    return  scores

In [21]:
import torch
from fastapi import FastAPI, Form , Request , Response
import uvicorn
from colabcode import ColabCode
from fastapi.templating import Jinja2Templates
from fastapi.responses import HTMLResponse
from fastapi.middleware.cors import CORSMiddleware
import io
import asyncio
import json
import base64

templates = Jinja2Templates(directory = "/content/")

app = FastAPI()

origins = [
    "http://localhost",
    "http://localhost:8000"
    "http://localhost:8080",
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

TIME_OUT_ERROR = 20

@app.post("/calculate")
async def calculate(request : Request ) :

  data = await request.json()
  print(data)
  print("...................... inside the  model ............ ")

  prompt = data['prompt']
  cnt = int(data['N'] )


  img = await getImages(gen_model , prompt , cnt )
  scores = await scoreImages(asthetic_model  , clip_model , img )
  print(".................The image output is this .......................")

  img_arr = []

  l = []
  for i in range(cnt) :

    img_byte_arr = io.BytesIO()
    scores[i][1].save(img_byte_arr, format='PNG')
    img_byte = base64.b64encode(img_byte_arr.getvalue())
    img_byte = "data:image/png;base64," + img_byte.decode('utf-8')
    obj = {"image": img_byte , "score" : scores[i][0]}
    l.append(obj)

  return json.dumps(l)

@app.get("/" , response_class = HTMLResponse)
def home(request : Request ) :

  return templates.TemplateResponse("index.html" , {"request": request })

In [26]:
server = ColabCode(port = 8000, code=False )
server.run_app(app=app)

Public URL: NgrokTunnel: "https://4c4f-34-147-60-144.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [1153]
INFO:uvicorn.error:Started server process [1153]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2601:646:ca82:f20:ce8:138e:fef1:9161:0 - "GET / HTTP/1.1" 200 OK
INFO:     2601:646:ca82:f20:ce8:138e:fef1:9161:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
{'prompt': 'blu blu blad', 'N': '2'}
...................... inside the  model ............ 
.................Generating images ...............


  0%|          | 0/51 [00:00<?, ?it/s]

............Scoring images.................


...

............Scoring images.................


100%|██████████| 2/2 [00:00<00:00, 52.86it/s]

.................The image output is this .......................


INFO:     2601:646:ca82:f20:ce8:138e:fef1:9161:0 - "POST /calculate HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:uvicorn.error:Shutting down
INFO:     Waiting for application shutdown.
INFO:uvicorn.error:Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:uvicorn.error:Application shutdown complete.
INFO:     Finished server process [1153]
INFO:uvicorn.error:Finished server process [1153]


In [24]:
!npx kill-port 8000

npx: installed 3 in 2.202s
Could not kill process on port 8000. No process running on port.


In [25]:
!killall ngrok

ngrok: no process found
